## Abertura e Tratamento do Arquivo
- Abri o arquivo .txt
- Extrair a quantiade de processadores do Servidor
- Tratar a entrada de dados

In [89]:
with open("Entrada de Arquivos/caso100.txt") as arquivo:
    caso = arquivo.readlines()

cpu = int(str(caso[0])[7:])
tasks = list(map(lambda x: x[:-1], caso[1:]))

print(f"Numero de Processadores: {cpu}")
print(f"Lista de Taske: {tasks}")

Numero de Processadores: 12
Lista de Taske: ['mqpd_22 -> vbkdb_311', 'mqpd_22 -> wvq_257', 'mqpd_22 -> njll_448', 'vbkdb_311 -> wfsp_65', 'vbkdb_311 -> xy_208', 'wvq_257 -> wwm_441', 'fnna_218 -> mqpd_22', 'fnna_218 -> jcolf_337', 'fnna_218 -> kduy_273', 'fnna_218 -> iqhlm_9', 'fnna_218 -> jr_122', 'jcolf_337 -> tbiha_504', 'njll_448 -> nrtf_30', 'njll_448 -> riao_303', 'kduy_273 -> ourjc_112', 'iqhlm_9 -> jx_478', 'iqhlm_9 -> ko_232', 'jr_122 -> nwm_460', 'wfsp_65 -> womam_53', 'tbiha_504 -> ttc_386', 'nrtf_30 -> opv_80', 'dkqwb_420 -> fnna_218', 'riao_303 -> wv_309', 'riao_303 -> wsc_205', 'ourjc_112 -> qxs_274', 'ourjc_112 -> qvtrh_489', 'clech_394 -> dkqwb_420', 'clech_394 -> hup_449', 'clech_394 -> kyao_184', 'clech_394 -> om_482', 'jx_478 -> onfv_345', 'jx_478 -> nmiua_347', 'bg_40 -> clech_394', 'ko_232 -> qalh_139', 'ko_232 -> of_315', 'nwm_460 -> ol_378', 'hup_449 -> ixr_500', 'hup_449 -> hx_169', 'opv_80 -> tpxd_218', 'afkqs_416 -> bg_40', 'kyao_184 -> lnc_225', 'wwm_441 -> x

## Criar um algoritimo Modificado de Arvore de Busca

- Criar a classe Node
- Criar um algoritimo de arvore convencional
- O algoritimo deve caminha utilizando recursividade e numero de CPU
- Definir duas politicas de caminhamento na arvore (maior número)-(menor número)
- Criar uma função de caminhamento para utilizar as duas politicas 
- Deve ter um contador de "Tempo" e exibir o tempo final do percurso

### Classe Node

In [90]:
class Node():
    def __init__(self, valor, tempo):
        self.valor = valor
        self.tempo_restante = tempo
        self.filhos = []
        self.pai = None
        self.free = False

    def adicionar_filho(self, filho):
        filho.pai = self
        self.filhos.append(filho)
        

### Class ProcessTree
- Classe de Arvore de Busca adaptado

In [91]:
class ProcessTree():
    def __init__(self):
        self.root = None
        self.nodes = {}

    def get_or_create(self, nome):
        if nome not in self.nodes:
            ident, tempo = nome.split("_")
            tempo = int(tempo)
            self.nodes[nome] = Node(ident, tempo)
        return self.nodes[nome]
    
    def add_relation(self, pai_nome, filho_nome):
        pai = self.get_or_create(pai_nome)
        filho = self.get_or_create(filho_nome)
        pai.adicionar_filho(filho)

    def find_root(self):
        for node in self.nodes.values():
            if node.pai is None:
                self.root = node
                node.free = True
                return node
        return None

    def insert(self, valor, tempo):
        new = Node(valor, tempo)
        if self.root is None:
            new.free = True
            self.root = new
            return
        fila = [self.root]
        while fila:
            node = fila.pop(0)
            if not node.filhos or valor < node.filhos[-1].valor:
                node.adicionar_filho(new)
                break
            else:
                fila.extend(node.filhos)

    def pre_execution(self, politica="MIN"):
        prontos = []
        fila = [self.root]
        while fila:
            node = fila.pop(0)
            if node.free and node.tempo_restante > 0:
                prontos.append(node)
            fila.extend(node.filhos)
        
        if politica == "MIN":
            return sorted(prontos, key=lambda x: x.valor)
        elif politica == "MAX":
            return sorted(prontos, key=lambda x: x.valor, reverse=True)
        else:
            raise ValueError("Política deve ser 'MIN' ou 'MAX'")

    
    def execution(self, cpu, politica="MIN"):
        total_time = 0
        while True:
            prontos = self.pre_execution(politica)
            if not prontos:
                break

            ativos = prontos[:cpu]
            menor_tempo = min(node.tempo_restante for node in ativos)
            total_time += menor_tempo

            for node in ativos:
                node.tempo_restante -= menor_tempo
                if node.tempo_restante == 0:
                    for filho in node.filhos:
                        filho.free = True

        return total_time

## Tratar a Lista de Tasks e prepara para inserir na Árvore
- Tratar retirando as "->" e transformando pais e filhos
- Setar a nova root

In [96]:
p_tree = ProcessTree()
pol = "MIN"

for famili in tasks:
    pai, filho = [p.strip() for p in famili.split("->")]
    p_tree.add_relation(pai, filho)

p_tree.find_root()

total_time = p_tree.execution(cpu=cpu, politica=pol)
print(f"Tempo total de execução na politica {pol}: {total_time}")

Tempo total de execução na politica MIN: 3315
